In [ ]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! nvidia-smi 
! nvidia-smi -L

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Input

#accuracy metric
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

def accuracy(train_x, train_y, test_x, test_y):
    cart_model = RandomForestClassifier(50)
    cart_model.fit(train_x, train_y)
    labels_pred = cart_model.predict(test_x)
    current_acc = accuracy_score(test_y, labels_pred) * 100
    return current_acc


In [ ]:
#set random seed
#GPU 0: Tesla V100-SXM2-16GB
import os
import random
import tensorflow as tf
import numpy as np

#check version
print('numpy',np.__version__)
print('tensorflow',tf.__version__)
!python --version
def set_seed(seed=784):

  os.environ['PYTHONHASHSEED']=str(0)
  random.seed(seed)
  tf.random.set_seed(seed)
  tf.keras.utils.set_random_seed(seed)
  tf.compat.v1.set_random_seed(seed)
  np.random.seed(seed)
  os.environ['TF_DETERMINISTIC_OPS'] = '0' 

seed='5'
set_seed(int(seed))
print(int(seed))

#seed:5,10,20,31,40,50,60,70,104,567,
      # 784,871,1002,1945,4000,3912,5678,40532,78,90
      # 43,56,563,25,34,64,66,18,34,27


In [ ]:
from tensorflow import keras
import tensorflow as tf
import os
import numpy as np

MNIST_PATH = "./mnist.npz"
def load_mnist(path):
    if os.path.isfile(path):
        with np.load(path, allow_pickle=True) as f:
            x_train, y_train = f['x_train'], f['y_train']
            x_test, y_test = f['x_test'], f['y_test']
        return (x_train, y_train), (x_test, y_test)
    return keras.datasets.mnist.load_data(MNIST_PATH)

def get_half_batch_ds(batch_size):
    return get_ds(batch_size//2)

def get_ds(batch_size):
    (x, y), (x_T,y_T) = load_mnist(MNIST_PATH)
    x = _process_x(x)
    x_T=_process_x(x_T)

    y = tf.cast(y, tf.int32)
    y_T = tf.cast(y_T, tf.int32)

    ds = tf.data.Dataset.from_tensor_slices((x, y)).cache().shuffle(1024).batch(batch_size) \
        .prefetch(tf.data.experimental.AUTOTUNE)
    #.cache 将preprocess的数据存储在缓存空间中将大幅提高计算速度
    #prefetch 提速
    return ds,x_T.numpy(),y_T.numpy()
def _process_x(x):
    return tf.expand_dims(tf.cast(x, tf.float32), axis=3) / 255. * 2 - 1

# MSE GAN

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

def _img_recenter(img):
    return (img + 1) * 255 / 2

def _save_gan(model_name, ep, t,imgs, seed,show_label=False, nc=10, nr=10):
    if not isinstance(imgs, np.ndarray):
        imgs = imgs.numpy()
    if imgs.ndim > 3:
        imgs = np.squeeze(imgs, axis=-1)
    imgs = _img_recenter(imgs)
    plt.clf()
    plt.figure(0, (nc * 2, nr * 2))
    for c in range(nc):
        for r in range(nr):
            i = r * nc + c
            plt.subplot(nr, nc, i + 1)
            plt.imshow(imgs[i], cmap="gray")
            plt.axis("off")
            if show_label:
                plt.text(23, 26, int(r), fontsize=23)
    plt.tight_layout()
    dir_ = "drive/My Drive/VGAN/"+seed+"/visual_mse_"+seed
    os.makedirs(dir_, exist_ok=True)
    path = dir_ + "/"+str(ep)+"_"+str(t)+".png"
    plt.savefig(path)

def save_gan(model, ep,t,seed, **kwargs):
    name = model.__class__.__name__.lower()
    if name in ["dcgan", "wgan", "wgangp", "lsgan", "wgandiv", "sagan", "pggan"]:
        imgs = model.call(100, training=False).numpy()
        _save_gan(name, ep, imgs, show_label=False)
    elif name == "gan":
        data = model.call(5, training=False).numpy()
        plt.plot(data.T)
        plt.xticks((), ())
        dir_ = "visual/{}".format(name)
        os.makedirs(dir_, exist_ok=True)
        path = dir_ + "/{}.png".format(ep)
        plt.savefig(path)
    elif name == "cgan" or name == "acgan":
        img_label = np.arange(0, 10).astype(np.int32).repeat(10, axis=0)
        imgs = model.predict(img_label)
        _save_gan(name, ep, t,imgs, seed,show_label=True)

    else:
        raise ValueError(name)
    plt.clf()
    plt.close()



import tensorflow as tf
import os

_b_acc = None
_c_acc = None

def set_soft_gpu(soft_gpu):
    if soft_gpu:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
def binary_accuracy(label, pred):
    global _b_acc
    if _b_acc is None:
        _b_acc = tf.keras.metrics.BinaryAccuracy()
    _b_acc.reset_states()
    _b_acc.update_state(label, pred)
    return _b_acc.result()

def save_weights(model,ep,t,seed):
    os.makedirs("drive/My Drive/VGAN/"+seed+"/Model_mse_"+seed+"/", exist_ok=True)
    model.save_weights("drive/My Drive/VGAN/"+seed+"/Model_mse_"+seed+"/Model_"+str(ep)+"_"+str(t)+"/Model.ckpt")

#define V-method
from numba import jit

@jit()
def Vmatrix_np(x):
    v_matrix=np.zeros((x.shape[0],x.shape[0]))
    ve=np.zeros((1,x.shape[1]))
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            for m in range(x.shape[1]):
                xt = x[:, m]
                ve[0,m] = np.subtract(np.max(xt),np.maximum(xt[i], xt[j]))
            v=np.sum(ve)
            v_matrix[i,j]=v
             
    return v_matrix
def vloss_tf_bigan(x_r, x_f, v):
    dif = x_f - x_r
    loss = tf.matmul(tf.matmul(dif, v, transpose_a=True), dif)
    return loss


from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense, Reshape, Conv2DTranspose, ReLU, BatchNormalization, LeakyReLU
from tensorflow import keras
import tensorflow as tf

def mnist_uni_disc_cnn(input_shape=(28, 28, 1), use_bn=True):
    model = keras.Sequential()
    # [n, 28, 28, n] -> [n, 14, 14, 64]
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', input_shape=input_shape))
    if use_bn:
        model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    # -> [n, 7, 7, 128]
    model.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    if use_bn:
        model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.3))
    model.add(Flatten())
    return model

def mnist_uni_gen_cnn(input_shape):
    return keras.Sequential([
        # [n, latent] -> [n, 7 * 7 * 128] -> [n, 7, 7, 128]
        Dense(7 * 7 * 64, input_shape=input_shape),
        BatchNormalization(),
        ReLU(),
        Reshape((7, 7, 64)),
        # -> [n, 14, 14, 64]
        Conv2DTranspose(32, (4, 4), strides=(2, 2), padding='same'),#channel=64,kernelsize(4,4)
        BatchNormalization(),
        ReLU(),
        # -> [n, 28, 28, 32]
        Conv2DTranspose(16, (4, 4), strides=(2, 2), padding='same'),
        BatchNormalization(),
        ReLU(),
        # -> [n, 28, 28, 1]
        Conv2D(1, (4, 4), padding='same', activation=keras.activations.tanh)
    ])
# [Conditional Generative Adversarial Nets](https://arxiv.org/pdf/1411.1784.pdf)
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Dense, Reshape, Input, Embedding
import time

class CGAN(keras.Model):
    """
    discriminator 标签+图片 预测 真假
    generator 标签 生成 图片
    """
    def __init__(self, latent_dim, label_dim, img_shape):
        #全局设置
        super().__init__()
        self.latent_dim = latent_dim #设置潜在空间的维度大小
        self.label_dim = label_dim
        self.img_shape = img_shape

        self.g = self._get_generator()
        self.d = self._get_discriminator()

        self.opt = keras.optimizers.Adam(0.0002, beta_1=0.5)
        self.loss_func = keras.losses.BinaryCrossentropy(from_logits=True)

    def call(self, target_labels, training=None, mask=None):
        noise = tf.random.normal((len(target_labels), self.latent_dim))
        if isinstance(target_labels, np.ndarray):
            target_labels = tf.convert_to_tensor(target_labels, dtype=tf.int32)
        return self.g.call([noise, target_labels], training=training)
      
    def d_loss_wasserstein(self, real_logits, fake_logits):
        d_loss = tf.reduce_mean(fake_logits) - tf.reduce_mean(real_logits)

        return d_loss

    def g_loss_wasserstein(self, fake_logits):
        g_loss = -tf.reduce_mean(fake_logits)

        return g_loss

    def wasserstein_gradient_penalty(self, x, x_fake, labels):
        # temp_shape = [x.shape[0]]+[1 for _ in  range(len(x.shape)-1)]
        epsilon = tf.random.uniform([], 0.0, 1.0)
        x_hat = epsilon * x + (1 - epsilon) * x_fake

        with tf.GradientTape() as t:
            t.watch(x_hat)
            d_hat = self.d([x_hat, labels], training=False)
        gradients = t.gradient(d_hat, x_hat)
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients)))
        gradient_penalty = 1 * tf.reduce_mean((slopes - 1.0) ** 2)

        return gradient_penalty

    def _get_discriminator(self):
        img = Input(shape=self.img_shape)
        label = Input(shape=(), dtype=tf.int32)
        label_emb = Embedding(10, 32)(label)
        emb_img = Reshape((28, 28, 1))(Dense(28*28, activation=keras.activations.relu)(label_emb))
        concat_img = tf.concat((img, emb_img), axis=3)
        s = keras.Sequential([
            mnist_uni_disc_cnn(input_shape=[28, 28, 2]),
            Dense(1)
        ])
        o = s(concat_img)
        model = keras.Model([img, label], o, name="discriminator")
        model.summary()
        return model

    def _get_generator(self):
        noise =  Input(shape=(self.latent_dim,))
        label = Input(shape=(), dtype=tf.int32)
        label_onehot = tf.one_hot(label, depth=self.label_dim)
        model_in = tf.concat((noise, label_onehot), axis=1)
        s = mnist_uni_gen_cnn((self.latent_dim+self.label_dim,))
        o = s(model_in)
        model = keras.Model([noise, label], o, name="generator")
        model.summary()
        return model
# d_loss, d_acc = self.train_d(img, gen_img,img_label,gen_img_label)
    def train_d(self, img,gen_img, img_label):
        with tf.GradientTape() as tape:
            real_logits = self.d.call([img, img_label], training=True)
            fake_logits = self.d.call([gen_img, img_label], training=True)

            # loss = self.loss_func(pred)
            loss=self.d_loss_wasserstein(real_logits,fake_logits)
            gp=self.wasserstein_gradient_penalty(img,gen_img,img_label)
            gloss=loss
        grads = tape.gradient(gloss, self.d.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.d.trainable_variables))
        return gloss

    def train_g(self, real_img,random_img_label):
        d_label = tf.ones((len(random_img_label), 1), tf.float32)   # let d think generated images are real
        with tf.GradientTape() as tape:
            g_img = self.call(random_img_label, training=True)
            fake_logits = self.d.call([g_img, random_img_label], training=False)
            #wloss
            loss=self.g_loss_wasserstein(fake_logits)
            # kl loss
            # loss = self.loss_func(d_label, pred)

            #mse loss
            loss_mse=tf.reduce_mean(tf.losses.mean_squared_error(real_img,g_img))
            g_loss=loss+loss_mse
        grads = tape.gradient(g_loss, self.g.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.g.trainable_variables))
        return g_loss, g_img,loss_mse

    def step(self, real_img, real_img_label):
        g_loss, g_img ,loss_mse= self.train_g(real_img,real_img_label)

        gen_img=g_img[:len(g_img)]
        img = real_img

        img_label =real_img_label

        d_loss= self.train_d(img, gen_img,img_label)
        return g_img, d_loss, g_loss,loss_mse


def train(gan, ds,x_T,y_T,seed):
    t0 = time.time()

    loss_G=[]
    loss_D=[]
    acc_value=[]

    acc=0
    for ep in range(EPOCH):
        for t, (real_img, real_img_label) in enumerate(ds):
            # print(real_img_label.shape)
            g_img, d_loss, g_loss,loss_mse = gan.step(real_img, real_img_label)
            loss_G.append(loss_G)
            loss_D.append(loss_D)
            y_t = np.arange(0, 10).astype(np.int32).repeat(32, axis=0)
            x_t = gan.predict(y_t)

            x_t=x_t.reshape(-1,28*28*1)
            x_T=x_T.reshape(-1,28*28*1)

            current_acc=accuracy(x_t,y_t,x_T,y_T)

            if current_acc>acc:
              acc=current_acc
              t1 = time.time()
              acc_value.append([current_acc,t])
              print("ep={} | time={:.2f} | t={}  | d_loss={:.5f} | g_loss={:.5f}| mse_loss={:.5f}".format(
              ep, t1-t0, t, d_loss.numpy(), g_loss.numpy(),loss_mse.numpy() ))
              save_gan(gan,ep, t,seed)
              save_weights(gan,ep,t,seed)
              print('The best acccuracy is:',current_acc)

    print('asdasdad')
    y_t = np.arange(0, 10).astype(np.int32).repeat(32, axis=0)
    x_t = m.predict(y_t)
    x_t=x_t.reshape(-1,28*28*1)
    x_T=x_T.reshape(-1,28*28*1)

    last_acc=accuracy(x_t,y_t,x_T,y_T)
    save_gan(gan,ep, t,seed)
    save_weights(gan,ep,t,seed)
    print('The last accuracy is:',last_acc)

    t_end=time.time()
    print('total running :',t_end-t0)

#run
LATENT_DIM = 100
IMG_SHAPE = (28, 28, 1)
LABEL_DIM = 10
BATCH_SIZE = 64
EPOCH = 5

set_soft_gpu(True)
d,x_T,y_T = get_half_batch_ds(BATCH_SIZE)
m = CGAN(LATENT_DIM, LABEL_DIM, IMG_SHAPE)
train(m, d,x_T,y_T,seed)

In [ ]:
# ep=5
# t=1533
# dir='drive/Shared drives/Ziqiang/VGAN/Model_mse/'
# checkpoint_dir=dir+'Model_'+str(ep)+'_'+str(t)+'/Model.ckpt'
# print(checkpoint_dir)
# m.load_weights(checkpoint_dir)

# img_label = np.arange(0, 10).astype(np.int32).repeat(32, axis=0)
# imgs = m.predict(img_label)

# print(imgs.shape)
# # #visual
# imgs = np.squeeze(imgs, axis=-1)
# imgs = _img_recenter(imgs)
# for i in range(10): 
#   plt.figure(i)
#   plt.imshow(imgs[32*i+3], cmap="gray")